# <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#37FABC; font-size:160%; text-align:center;padding: 0px; border-bottom: 5px solid #407A68">Playground Series S3-E6 EDA and modeling</p>

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 5px solid #008F77">Intro</p>

**🟦EN**: This Kaggle workbook aims to provide a comprehensive exploratory data analysis (EDA) and a set of simple models (which will not be optimized), but which can give a vague idea of how to choose the best model for the given data set, with the ultimate goal of making decisions.

Through this EDA, we will be able to get a deeper understanding of the structure of the data, the values that have a relationship between them and the missing values and pattern or outliers that may affect when performing the modeling or selecting the model we want to use for prediction/recommendation. By performing an EDA, we can identify potential pitfalls and make the decisions and subsequent processing necessary to improve the performance and accuracy of the models. **

**🟥ES**: Este cuaderno Kaggle tiene el objetivo proporcionar un análisis exploratorio de datos (AED) exhaustivo y un conjunto de modelos simples (los cuales no estarán optimizados), pero que pueden llegar a dar una vaga idea para escoger el mejor modelo, para el conjunto de datos dado, con el objetivo final de tomar decisiones.

A través de este AED, podremos obtener una comprensión más profunda de la estructura de los datos, los valores que tiene una relación entre ellos y los valores que faltan y patrón o valores anómalos que pueda afectar a la hora de realizar el modelado o seleccionar el modelo que queremos utilizar para la predicción / recomendación. Al realizar un EDA, podemos identificar posibles obstáculos y tomar las decisiones, y posteriormente el procesado necesario para mejorar el rendimiento y la precisión de los modelos.

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Useful information</p>
<p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#2C3FD6; font-size:130%; text-align:left;padding: 0px;"><strong>Playground series S3-E6 Dataset</strong></p>

**🟦EN**: 

* *train.csv* - the training dataset; price is the target
* *test.csv* - the test dataset; your objective is to predict price
* *sample_submission.csv* - a sample submission file in the correct format

**🟥ES**:

* *train.csv* - el conjunto de datos de entrenamiento; el precio es el objetivo
* *test.csv* - el conjunto de datos de prueba; su objetivo es predecir el precio
* *sample_submission.csv* - un archivo de envío de muestra en el formato correcto

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Libraries</p>

In [ ]:
import os
import sys
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import cohen_kappa_score, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer

In [ ]:
# Put theme of notebook 
from colorama import Fore, Style

# Colors
red = Fore.RED + Style.BRIGHT
mgta = Fore.MAGENTA + Style.BRIGHT
yllw = Fore.YELLOW + Style.BRIGHT
cyn = Fore.CYAN + Style.BRIGHT
blue = Fore.BLUE + Style.BRIGHT

# Reset
res = Style.RESET_ALL

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Constants</p>

In [ ]:
PATH = "/kaggle/input/playground-series-s3e6"
ORIGINAL_DATASET = "/kaggle/input/paris-housing-price-prediction/ParisHousing.csv"

TEST_FILENAME = "test.csv"
TRAIN_FILENAME = "train.csv"
SAMPLE_SUBMISSION_FILENAME = "sample_submission.csv"

TRAIN_DIR = os.path.join(PATH, TRAIN_FILENAME)
TEST_DIR = os.path.join(PATH, TEST_FILENAME)
SAMPLE_SUBMISSION_DIR = os.path.join(PATH, SAMPLE_SUBMISSION_FILENAME)


# Values
SQUERE_METERS = "squareMeters"
NUMBER_ROOMS = "numberOfRooms"
HAS_YARD = "hasYard"
HAS_POOL = "hasPool"
FLOORS = "floors"
CITY_CODE = "cityCode"
CITY_PART_RANGE = "cityPartRange"
NUM_PREV_OWNERS = "numPrevOwners"
MADE = "made"
IS_NEW_BUILT = "isNewBuilt"
HAS_STORM_PROTECTOR = "hasStormProtector"
BASMENT = "basement"
ATTIC = "attic"
GARAGE = "garage"
HAS_STORAGE_ROOM = "hasStorageRoom"
HAS_GUESS_ROOM = "hasGuestRoom"
TARGET = "price"
ID = "id"

# Colors
CREME_CYAN = "#93adad"
MAGENTA = "#f78dad"
DARK_BLUE = "#8084bd"
CREME = "#DEB078"
PINK = "#e28ee8"
ORANGE = "#e8a18e"
YELLOW = "#e8e78e"
PURLPLE = "#bb8ee8"
COLORS = [CREME_CYAN, MAGENTA, DARK_BLUE, CREME, PINK, ORANGE, YELLOW, PURLPLE]
print(TEST_DIR)

In [ ]:
CATEGORICAL_COLS = [HAS_YARD, HAS_POOL,HAS_STORM_PROTECTOR, 
                    HAS_STORAGE_ROOM, HAS_GUESS_ROOM, IS_NEW_BUILT,
                    CITY_CODE, CITY_PART_RANGE]

NUM_COLS = [SQUERE_METERS, NUMBER_ROOMS, FLOORS,
            NUM_PREV_OWNERS, BASMENT, ATTIC, GARAGE]

FEATURE_COLS = CATEGORICAL_COLS + NUM_COLS

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Functions</p>

In [ ]:
def data_description(df):
    print("Data description")
    print(f"Total number of records {df.shape[0]}")
    print(f'number of features {df.shape[1]}\n\n')
    columns = df.columns
    data_type = []
    
    # Get the datatype of features
    for col in df.columns:
        data_type.append(df[col].dtype)
        
    n_uni = df.nunique()
    # Number of NaN values
    n_miss = df.isna().sum()
    
    names = list(zip(columns, data_type, n_uni, n_miss))
    variable_desc = pd.DataFrame(names, columns=["Name","Type","Unique levels","Missing"])
    print(variable_desc)


In [ ]:
def show_corr_heatmap(df, title):
    
    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True

    plt.figure(figsize = (15, 10))
    plt.title(title)
    sns.heatmap(corr, annot = True, linewidths=.5, fmt=".2f", square=True, mask = mask)
    plt.show()

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Import data</p>

In [ ]:
train_df = pd.read_csv(TRAIN_DIR, dtype={CITY_CODE: object})
test_df = pd.read_csv(TEST_DIR, dtype={CITY_CODE: object})
original_df = pd.read_csv(ORIGINAL_DATASET, dtype={CITY_CODE: object})
original_df = original_df.reset_index()
original_df[ID] = original_df["index"] + 1000000
original_df = original_df.drop(columns = ["index"])
train_df.head()

In [ ]:
original_df.head()

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">EDA and data modification</p>

In [ ]:
train_df[CITY_CODE] = train_df[CITY_CODE].str.zfill(5)
test_df[CITY_CODE] = train_df[CITY_CODE].str.zfill(5)
original_df[CITY_CODE] = train_df[CITY_CODE].str.zfill(5)

In [ ]:
print(f"Train dataframe length: {mgta}{len(train_df)}{res}\tTest dataframe length: {mgta}{len(test_df)}{res}\tOriginal dataframea length: {mgta}{len(original_df)}{res}")

In [ ]:
num_cols = train_df.select_dtypes(include=np.number).columns.tolist()
num_cols.remove(ID)
num_cols.remove(TARGET)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 5))

sns.histplot(original_df[TARGET], color="red", ax=ax[0], kde=True)
ax[0].set_title("Price distribution (Original Dataset)", fontsize=12)
for patch in ax[0].patches:
    patch.set_fc(MAGENTA)
    
sns.histplot(train_df[TARGET], color="blue", ax=ax[1], kde=True)
ax[1].set_title("Price distribution (Playground Dataset)", fontsize=12)
for patch in ax[1].patches:
    patch.set_fc(CREME_CYAN)
    
plt.tight_layout()
plt.show()

In [ ]:
ncols = 4
nrows = int(np.ceil(len(num_cols)/ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 15))
for ax, x in zip(axes.flat, num_cols):
    sns.kdeplot(train_df[x], color="#FC5154", label="train", ax=ax)
    sns.kdeplot(test_df[x], color="#B3B248", label="test", ax=ax)
    sns.kdeplot(original_df[x], color="#83CFFC", label="original", ax=ax)
    ax.set_title(f"Distribution of {x}")
    ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
total_df = pd.concat([train_df, test_df], axis=0)
total_df['set'] = 'test '
total_df.iloc[:train_df.shape[0], -1] = 'train'

fig, ax = plt.subplots(2, 2, figsize=(17, 10))

sns.scatterplot(ax=ax[0,0], data=train_df, x=TARGET,y=SQUERE_METERS, color=CREME_CYAN)
ax[0, 0].set_title(f"{TARGET} / {SQUERE_METERS} train dataframe")
sns.scatterplot(ax=ax[0,1], data=original_df, x=TARGET,y=SQUERE_METERS, color=DARK_BLUE)
ax[0, 1].set_title(f"{TARGET} / {SQUERE_METERS} original dataframe")
sns.boxplot(y=total_df[SQUERE_METERS], x=total_df['set'], ax=ax[1][0])
sns.boxplot(y=total_df[GARAGE], x=total_df['set'], ax=ax[1][1], color=PINK)

In [ ]:
sns.boxplot(y = TARGET, data = train_df)

In [ ]:
display(show_corr_heatmap(train_df, "Train correlation heatmap"))
display(show_corr_heatmap(test_df, "Train correlation heatmap"))
display(show_corr_heatmap(original_df, "Train correlation heatmap"))

In [ ]:
made = pd.DataFrame(train_df.groupby([MADE]).size()).reset_index().rename(columns = {0:"Count"})
made = made[made[MADE]< 5000]

made1 = pd.DataFrame(train_df.groupby([MADE])[TARGET].mean()).reset_index().rename(columns = {0:"mean_price"})

made = pd.merge(made,made1, on = MADE, how = "inner")

ax = made.plot(x=MADE, y="Count", legend=False)
ax2 = ax.twinx()
made.plot(x=MADE, y=TARGET, ax=ax2, legend=False, color=MAGENTA)
ax.figure.legend()
plt.show()

In [ ]:
made[made[MADE]<= 2000].corr()

In [ ]:
made[made[MADE]> 2007].corr()

In [ ]:
train_df = pd.concat([train_df, original_df])

In [ ]:
train_df.head()

In [ ]:
# train_1 = train_df[train_df[MADE]<=2000]
# test_1 = test_df[test_df[MADE]<=2000]
# train_2 = train_df[(train_df[MADE]>=2001) & (train_df[MADE]<=2007)]
# test_2 = test_df[(test_df[MADE]>=2001) & (test_df[MADE]<=2007)]
# train_3 = train_df[(train_df[MADE]>=2008) & (train_df[MADE]<=2015)]
# test_3 = test_df[(test_df[MADE]>=2008) & (test_df[MADE]<=2015)]
# train_4 = train_df[train_df[MADE]>2015]
# test_4 = test_df[test_df[MADE]>2015]
train_1 = train_df[train_df[MADE]<=2000]
test_1 = test_df[test_df[MADE]<=2000]
train_2 = train_df[(train_df[MADE]>=2001) & (train_df[MADE]<=2003)]
test_2 = test_df[(test_df[MADE]>=2004) & (test_df[MADE]<=2007)]
train_3 = train_df[(train_df[MADE]>=2008) & (train_df[MADE]<=2015)]
test_3 = test_df[(test_df[MADE]>=2008) & (test_df[MADE]<=2015)]
train_4 = train_df[train_df[MADE]>2015]
test_4 = test_df[test_df[MADE]>2015]

In [ ]:
train_1 = pd.merge(train_1,made[[MADE,'Count']], on = MADE, how = 'inner')
test_1 = pd.merge(test_1,made[[MADE,'Count']], on = MADE, how = 'inner')
display(show_corr_heatmap(test_1, "Test_1 correlation heatmap"))
train_2 = pd.merge(train_2,made[[MADE,'Count']], on = MADE, how = 'inner')
test_2 = pd.merge(test_2,made[[MADE,'Count']], on = MADE, how = 'inner')
display(show_corr_heatmap(test_2, "Test_2 correlation heatmap"))
train_3 = pd.merge(train_3,made[[MADE,'Count']], on = MADE, how = 'inner')
test_3 = pd.merge(test_3,made[[MADE,'Count']], on = MADE, how = 'inner')
display(show_corr_heatmap(test_3, "Test_3 correlation heatmap"))
train_4 = pd.merge(train_4,made[[MADE,'Count']], on = MADE, how = 'inner')
test_4 = pd.merge(test_4,made[[MADE,'Count']], on = MADE, how = 'inner')
display(show_corr_heatmap(test_4, "Test_4 correlation heatmap"))

In [ ]:
num_cols = num_cols + ["Count"]

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Data preparation and prediction</p>

### <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#A766FA; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #5F06D1">XGBoost Model</p>

In [ ]:
LR = 0.24
NE = 2000

In [ ]:
X1 = train_1[num_cols]
# X1 = train_1[['squareMeters']]
y1 = train_1[TARGET]

model=XGBRegressor(max_depth=3, learning_rate=LR , n_estimators=NE, objective="reg:linear", booster="gbtree")
XGB=model.fit(X1,y1)


X_test = test_1[num_cols]
# X_test = test_1[['squareMeters']]
prediction=XGB.predict(X_test)

test_1[TARGET] = prediction
test_1

In [ ]:
X2 = train_2[num_cols]
# X2 = train_2[['squareMeters']]
y2 = train_2[TARGET]

model=XGBRegressor(max_depth=3, learning_rate=LR , n_estimators=NE, objective="reg:linear", booster="gbtree")
XGB=model.fit(X2,y2)

X_test = test_2[num_cols]
# X_test = test_2[['squareMeters']]
prediction=XGB.predict(X_test)

test_2[TARGET] = prediction
test_2

In [ ]:
X3 = train_3[num_cols]
y3 = train_3[TARGET]

model=XGBRegressor(max_depth=3, learning_rate=LR , n_estimators=NE, objective="reg:linear", booster="gbtree")
XGB=model.fit(X3,y3)

X_test = test_3[num_cols]
prediction=XGB.predict(X_test)

test_3[TARGET] = prediction
test_3

In [ ]:
X4 = train_4[num_cols]
y4 = train_4[TARGET]

model=XGBRegressor(max_depth=3, learning_rate=LR , n_estimators=NE, objective="reg:linear", booster="gbtree")
XGB=model.fit(X4,y4)

X_test = test_4[num_cols]
prediction=XGB.predict(X_test)

test_4[TARGET] = prediction
test_4

## <p style="font-family:Consolas Mono; font-weight:normal; letter-spacing: 2px; color:#06D1C7; font-size:130%; text-align:left;padding: 0px; border-bottom: 3px solid #008F77">Send prediction</p>

In [ ]:
submission = pd.concat([test_1, test_2, test_3, test_4])
submission = submission[[ID,TARGET]]
submission = submission.sort_values(by = [ID])
submission.to_csv("submission.csv",index = False)

In [ ]:
print(f"Train df {len(train_df)} train_3 len {len(train_3)}")

In [ ]:
submission.head()